In [66]:
import chromadb
import pandas as pd
from langchain.embeddings import GPT4AllEmbeddings

gpt4all_embd = GPT4AllEmbeddings()

client_chroma = chromadb.PersistentClient(path="db/")

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [122]:
import os
import psycopg2

# Get the database credentials from environment variables
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")

# Connect to the PostgreSQL database
connection = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)

cursor = connection.cursor()

SQL_avg_rating = "SELECT movieId, SUM(RATING)/COUNT(*) as AVG_RATING FROM ratings GROUP BY movieId"

SQL_tags_cat = "SELECT movieId, STRING_AGG(tag, ' ') FROM tags GROUP BY movieId"

SQL_movies = "SELECT movieId as mainMovieId, title, genres FROM movies"

QUERY = f"""
SELECT mvTcat.title, mvTcat.genres, rate.AVG_RATING, mvTcat.STRING_AGG
FROM
(
    ({SQL_movies}) mv
    JOIN ({SQL_tags_cat}) tcat
    ON mv.mainMovieId = tcat.movieId
) mvTcat
JOIN ({SQL_avg_rating}) rate
ON mvTcat.mainMovieId = rate.movieId;
"""

cursor.execute(SQL_avg_rating)
response = cursor.fetchall()
len(response)

# item_to_be_saved = {title} {genre} {rating} {tag1} {tag2}....{tagN}

9724

In [72]:
bak = response.copy()

In [85]:
final_data = list(map(lambda x: " ".join([str(k) for k in list(x)]), bak))

In [89]:
client_chroma.create_collection('movies')

Collection(name=movies)

In [90]:
from tqdm import tqdm

idX = 0
for data in tqdm(final_data):

    # Insert item_to_be_saved into the embedding engine 
    doc_result = gpt4all_embd.embed_query(data)
    doc_result

    # Save the embedded document to Chroma
    collection = client_chroma.get_collection('movies')

    # Use the following code as an example
    collection.add(
        embeddings =[doc_result],
        documents = [data],
        metadatas = [{"source": "batch"}],
        ids = [str(idX)]
    )

    idX += 1

100%|██████████| 48287/48287 [12:16<00:00, 65.58it/s]


In [115]:
collection = client_chroma.get_collection('movies')
data = "recommend movies with 5 stars" 
doc_result = gpt4all_embd.embed_query(data)
doc_result

collection.query(
    query_embeddings=doc_result,
    n_results=4
)

{'ids': [['6253', '1586', '6346', '912']],
 'distances': [[0.8868856430053711,
   0.9187239408493042,
   0.9367409944534302,
   0.9552434682846069]],
 'metadatas': [[{'source': 'batch'},
   {'source': 'batch'},
   {'source': 'batch'},
   {'source': 'batch'}]],
 'embeddings': None,
 'documents': [['Munich (2005) Action|Crime|Drama|Thriller 5.0 In Netflix queue',
   'Munich (2005) Action|Crime|Drama|Thriller 4.5 In Netflix queue',
   'Munich (2005) Action|Crime|Drama|Thriller 3.5 In Netflix queue',
   'Munich (2005) Action|Crime|Drama|Thriller 4.0 In Netflix queue']],
 'uris': None,
 'data': None}